In [1]:
# Βιβλιοθήκες
import pandas as pd
import numpy as np
import gekko as gk
import seaborn as sns
import xarray as xr
from matplotlib import pyplot as plt
import typing
import itertools
import math
import functools
from pca import pca
from IPython.display import display, HTML
import prince
import networkx
import sklearn
import tensorflow as tf
from tensorflow import keras
import multiprocessing
from imblearn.over_sampling import ADASYN, SMOTE
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import product
import arviz as az
import aesara
from factor_analyzer import FactorAnalyzer
import aesara.tensor as at
import scipy
import pymc as pm, pymc
import gpflow
from pymc.sampling.jax import sample_numpyro_nuts
import numpyro
import pickle

2022-12-10 11:46:19.455972: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64:
2022-12-10 11:46:19.456399: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64:
2022-12-10 11:46:19.456412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/media/alexander-fyrogenis/Elements/Διδακτορικό/Olive Oil/notebooks/evoos_env/lib/python3.10/

In [2]:
# Γενικές επιλογές εμφάνισης
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',30)

numpyro.set_platform("gpu")

figsize=(13,6)
fontsize=10
sns.set(rc={'figure.figsize':(15,10),
           'font.size':5})

In [3]:
# Utilities
from sklearn.preprocessing import StandardScaler

std_scale = lambda df: pd.DataFrame(data = StandardScaler().fit_transform(df), columns = df.columns, index=df.index)

def render_df(df:pd.DataFrame):
    from IPython.display import display, HTML
    display(HTML(df.to_html()))
    return

def full_display(r=None, c=None):
    pd.set_option('display.max_rows', r)
    pd.set_option('display.max_columns',c)
    return

def reset_display():
    pd.set_option('display.max_rows',10)
    pd.set_option('display.max_columns',30)

def full_display_once(df:pd.DataFrame, r=None, c=None):
    full_display(r=r, c=c)
    render_df(df)
    reset_display()

def rowwise_value_counts(df:pd.DataFrame):
    vcounts_df = pd.DataFrame(data = df.apply(lambda x: x.value_counts()).T.stack()).astype(int).T
    vcounts_df.index = ['']
    return vcounts_df

invert_dict = lambda e: {v:k for k, v in e.items()}

def categorical_scatter(X:pd.DataFrame, Y:typing.Optional[pd.DataFrame],cols:int=3,max_rows:int = 3,
                             figsize:tuple[int, int] = (30, 15), xaxis_label_size: int=12,
                             yaxis_label_size:int=12, categorical:typing.Optional[str]='hue')->typing.Optional[plt.figure]:
    '''
        Generate pair-wise scatter plots for a given DataFrame, with optional support for large Datasets
        and categorical variables. Yields a figure with `max_rows x cols` scatterplots per call. When
        `categorical=None` only generates pair-wise scatterplots for the columns of X, else generates all
        combinations of X-column pair and Y columns. When `categorical='hue'` values of the categorical Y
        variable will be depicted color-coded and when `categorical=size` different factors of the Y variable
        will have different sizes instead.
        
        Args:
        -----
        
            - X:pandas.DataFrame := The data to depict. When Y is also specified, X is assumed to be the DataFrame
            of indicator variables
            
            - Y:Optional[pandas.DataFrame] := Optional Dataframe of categorical variables to depict. Ignored if 
            `category` is `None`.
            
            - idx_lvl:int=1 := For multilevel indexed dataframes the lowest level to squash the index on. Must be non-negative
            
            - cols:int=3 := Number of columns for the resulting facet grid plot. Must be non-negative. Defaults to 3
            
            - max_rows:int=3 := Maximum number of rows per batch the generator yields. Defaults to 3 and must be non-negative
            
            - figsize:tuple[int, int] := A `height x width` tuple for the generated plots. Defaults to `(30, 15)`
            
            - xaxis_label_size:int=12 := The size of the x-axis titles for each subplot. Must be non-negative and defaults to 12
            
            - yaxis_label_size:int=12 := The size of the y-axis titles for each subplot. Must be non-negative and defaults to 12
            
            - categorical:Optional[str] := Set the display of the categorical variable. (1) `None` ignores `Y` and only displays X
            pair-wise scatterplots, (2) 'hue' displays the categorical variable of `Y` as color and (3) 'size' displays
            the categorical variable with differently sized points
            
        Returns:
        --------
        
            - fig:matplotlib.pyplot.figure := The figure object, a FacetPlot of scatterplots
            
       
       Raises:
        ------
        
            - WIP
            
            - ValueError 
    '''
    XY = pd.concat([Y, X], axis=1)
    idx_lvl= XY.columns.nlevels-1
    skip_multiindex = lambda df,i , idx_level=idx_lvl: df.columns[i][idx_level] if idx_level else df.columns[i]
    if idx_lvl:
        XY.columns = XY.columns.get_level_values(idx_lvl)
    x_combs = math.comb(X.shape[1], 2)
    MAX_ROWS = max_rows
    
    if categorical is not None:
        plots = x_combs*Y.shape[1]
    else:
        plots = x_combs
    ncols = cols
    size_scaling_factor = None
    total_rows = math.ceil(plots/ncols)
    total_figures = math.ceil(total_rows/MAX_ROWS)
    X_pairs = itertools.combinations(range(X.shape[1]), 2)
    subplots = itertools.product(X_pairs,range(Y.shape[1]) ) if categorical is not None else X_pairs
    exhaustion_sentinel = False
    while True:
        if exhaustion_sentinel: break
        fig, axs = plt.subplots(nrows=MAX_ROWS, ncols=ncols, figsize=figsize)
        ax_indices = itertools.product(range(MAX_ROWS), range(ncols), repeat=1)
        plotslice = itertools.islice(subplots, MAX_ROWS*ncols)
        fig_generator = itertools.zip_longest( ax_indices, plotslice, fillvalue = ((None, None), None) )
        for (axi, axj), e in fig_generator:
            e = tuple(flatten(e))
            if e[0] is not None:
                if categorical == 'size':
                    sns.scatterplot(x=skip_multiindex(X, e[0]), y=skip_multiindex(X, e[1]), data=XY,
                       size=skip_multiindex(Y, e[2]), ax=axs[axi, axj], legend=True)
                elif categorical == 'hue':
                    sns.scatterplot(x=skip_multiindex(X, e[0]), y=skip_multiindex(X, e[1]), data=XY,
                       hue=skip_multiindex(Y, e[2]), ax=axs[axi, axj], legend=True)
                elif categorical is None:
                    sns.scatterplot(x=skip_multiindex(X, e[0]), y=skip_multiindex(X, e[1]), data=XY,
                       ax=axs[axi, axj], legend=True)
                axs[axi, axj].set_xlabel(skip_multiindex(X, e[0]))
                axs[axi, axj].set_ylabel(skip_multiindex(X, e[1]))
                axs[axi, axj].xaxis.label.set_size(xaxis_label_size)
                axs[axi, axj].yaxis.label.set_size(yaxis_label_size)
                
            else:
                axs[axi, axj].axis('off')
                exhaustion_sentinel = True
        yield fig 
    return None



def flatten(xs):
    from collections.abc import Iterable
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

def tidy_multiindex(df:pd.DataFrame, sep:str="."):
    '''
        Compress a hierarchically indexed dataframe to standardized tidy
        format. A unique sepperator `sep` is used to allow reversal. All
        levels of the index are appended together with a delimeter to allow
        reversals.
        
        Args:
        ----
        
            - df:pandas.DataFrame := A `pandas.DataFrame` hierarchically indexed
            
            - sep:str='_._' := A delimenter delineating the different levels
            of the index. Ensure it is not present in any column name to avoid
            a malformed index
            
        Returns:
        --------
        
            - ndf:pandas.DataFrame := The DataFrame with a single-level index
    '''
    tidy_cols = (functools.reduce(lambda e1,e2: str(e1)+sep+str(e2), col ) for col in df.columns)
    ndf = df.copy(deep=True)
    ndf.columns = tidy_cols
    return ndf

def reverse_tidy_multiindex(df:pd.DataFrame, sep="."):
    '''
        Reverses the tidying to a hierachical format. Different
        levels of the index are identified based on "sep"
        
        Args:
        -----
        
            - df:pandas.DataFrame := The dataframe to process
            
            - sep:str='_._' := The string delimeter, sepperating
            values for different levels of the index
            
        Returns:
        -------
        
            - ndf:pandas.DataFrame := The dataframe with hierarchical index
    '''
    h_cols = (tuple(col.split(sep)) for col in df.columns)
    ndf = df.copy(deep=True)
    ndf.columns = pd.MultiIndex.from_tuples(h_cols)
    return ndf

def undummify(df:pd.DataFrame,cols:list[str, tuple[str]],ncol_name:typing.Union[str,tuple[str]],
              sep:typing.Optional[str]=None,
              rmap:typing.Optional[dict[int, typing.Union[str, tuple[str]]]]=None
             )->pd.DataFrame:
    '''
        Reverses hot-encoded variables in the DataFrame. A series of hot-encoded
        variable levels $(i_1, i2, \dots, i_k)$ is mapped to a single new column
        $(k)$, whose name is specified by `ncol_name`, in the new dataframe. Pre
        vious level columns are dropped.
        
        Args:
        ----
        
            - df:pandas.DataFrame := The DataFrame to operate upon
            
            - cols:list[str, tuple[str]] := A list of columns, representing the
            levels of a categorical variable
            
            - sep:Optional[str] := sepperator for variable level. Currently ignored
            
            - ncol_name:Union[str, tuple[str]] := Name of the new categorical column
            
            - remap:Optional[dict[int, Union[str, tuple[str]]]] := A dictionary mapping
            of categorical levels to values. Keys are the assumed to be levels, values
            are assumed to be values (i.e. strings). When provided, the previous levels
            will be replaced by the specified mappings in the new DataFrame
            
        Returns:
        -------
        
            - ndf:pandas.DataFrame := The processed dataframe
     '''
    _df = coredf.loc[:, cols]
    for i, col in enumerate(cols, 1):
        _df.loc[:, col] = i*_df.loc[:, col]
    ndf = df.copy(deep=True)
    ndf.drop(cols, axis=1, inplace=True)
    ndf[ncol_name] = _df.max(axis=1)
    c1 = df.columns.tolist()
    i = c1.index(cols[0])
    swp = ndf.columns.tolist()[:i-1]+[ndf.columns.tolist()[-1]]+ndf.columns.tolist()[i:-1]
    ndf = ndf.loc[:, swp]
    if rmap is not None:
        ndf = ndf.replace(rmap)
    return ndf
list_difference = lambda l1, l2: [e for e  in l1 if e not in set(l2)]

def corrspace_graph(df:pd.DataFrame):
    '''
        Generate a correlation graph representation of `df` datasets.
        Every node in the resulting graph is a variable and every 
        vertex between two nodes represents the correlation between
        the two variables, the correlation being the verted weight
        
        Args:
        ----
        
            - df:pandas.DataFrame := A dataset to depict
            
        Returns:
            - WIP
    '''
    
    return 


class DictTable(dict):
    '''
        Jupyter utility class that overrides the dicts' defaults
        __repr__ rendering the input dictionary to an HTML table
        for convenient jupyter rendering
    '''
    def _repr_html_(self):
        html = ["<table>"]
        for key, value in self.items():
            html.append("<tr>")
            html.append("<td>{0}</td>".format(key))
            html.append("<td>{0}</td>".format(value))
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
    
    
def boxplots(df:pd.DataFrame, max_features:int=20, title:str="Boxplot",
             **kwargs)->typing.Generator[plt.Figure, None, None]:
    '''
        Wrapper around `pandas.boxplot` for datasets with a large number
        of features. Returns a generator of boxplots of subsets of the total
        features.
        
        Args:
        -----
        
            - df:pandas.DataFrame := The DataFrame to plot
            
            - max_features:int>0 := The maximum number of features to plot
            in each generated boxplot
            
            - title:str='Boxplot' := The title of the plot. A subplot index
            of the form **(X/Y)**, tracking the current plot will be
            appended prior to rendering
            
            - **kwargs:dict[str:Any] := Keyword arguments to be be forwarded
            to pandas.DataFrame.boxplot
            
        Returns:
        -------
        
            - Generator[plt.Figure, None, None] := A generator yielding boxplots
            of feature subsets
    '''
    nplots = math.ceil(df.shape[1]/max_features)
    for i in range(nplots):
        if (i+1)*max_features<df.shape[1]:
            ndf = df.iloc[:,i*max_features:(i+1)*max_features]
        else:
            ndf = df.iloc[:,i*max_features:df.shape[1]]
        fig = ndf.boxplot(**kwargs)
        fig.set_title(title + " ({current}/{total})".format(current=i+1, total=nplots) )
        yield fig
        
def count_missing_nan(df:pd.DataFrame, axis:int=0):
    '''
        Return a new DataFrame with the counts of missing
        and invalid values across specified axis.
        
        Args:
        -----
        
            - df:pandas.DataFrame := The data
            
            - axis:int=0 := The axis across which missing
            values will be enumerated. Defaults to 0 (show
            missing values in each column)
            
        Returns:
        -------
        
            - missing_df:pd.DataFrame := A DataFrame containing
            counts of missing values
    '''
    ndf = pd.DataFrame(df.isna().sum(axis=axis)).T
    ndf.index = ['']
    return ndf


class PlotPrior:
    '''
        WIP
    '''
    def __init__(self,data, columns:typing.Optional[list[str, tuple[str]]]=None,
                 visible:bool=True, stack:bool=True, show_grid:bool=False, ncols:int=3,
                kde:bool=True):
        self.columns=columns
        self.visible=visible
        self.stack=stack
        self.ncols=ncols
        self.kde=kde
        self.plots={} 
    
    def __prep_data(self, data):
        stacked = data.prior.stack(samples=["chain", "draw"])
        if self.columns is None:
            self.columns = data.columns
        for col in self.columns:
            self.plots[col] = pd.DataFrame(data=stacked[col].values.T)
    
    def __call__(self, data, var:str):
        df = self.plots[var]
        k = df.shape[1]
        n = self.ncols
        m = (k - 1) // n + 1
        fig, axes = plt.subplots(nrows=m, ncols=n, figsize=(n * 5, m * 3))
        for i, (name, col) in enumerate(self.plots[var].items()):
            r, c = i // n, i % n
            ax = axes[r, c]
            col.hist(ax=ax)
            if self.kde:
                ax2 = col.plot.kde(ax=ax, secondary_y=True, title=name)
                ax2.set_ylim(0)
        fig.tight_layout()
        return fig,axs
    
def plot_kernel():
    '''
        Utility method for visualizing GP kernels. Produces a square visualization
        of the selected kernel and all possible combinations of hyperparameters
        
        TODO
        +++++
        
            - Extend this for combination kernels by nesting input hyperparameter
            dicts
            
            - Raise combinatronic explsion error when too many hyperparameter values
            are specified
        
        Args:
        ----
        
            - 
            
        Returns:
        --------
        
            -fig:matplotlib.pyplot.Figure := The generated figure
    '''
    
class MultiLayerPerceptronKernel(pymc.gp.cov.Covariance):
    '''
        Multi layer Perceptron Kernel a.k.a Arcsine Kernel
        Code transplanted from `GPy.kern.src.mlp`. This kernel
        is designed to mimic the highly nonlinear processes 
        of a Neural Network and can be thought of as the limit
        of an A.N.N. with a single input layer as the number
        of neurons in the hidden layer goes to infinity. Then
        the N.N., with normal priors over the weights and biases
        becomes equivalent to a G.P. This Kernel is given by:
        
        .. math::

          k(x,y) = \\sigma^{2}\\frac{2}{\\pi }  \\text{asin} \\left
          ( \\frac{ \\sigma_w^2 x^\\top y+\\sigma_b^2}{\\sqrt{\\sigma_w^2x^\\top x +
          \\sigma_b^2 + 1}\\sqrt{\\sigma_w^2 y^\\top y + \\sigma_b^2 +1}} \\right )
          
        Args:
        -----

            - input_dims:int=1 := The number of input dimentions (columns) used for Covariance
            computations. Defaults to 1 (use the first column only).

            - active_dims:Optional[list[int]]=None := None or array-like specifying which input
            dimentions will be used in computing the covariance matrix. Can be specified as a
            a boolean vector or a vector of indices. Optional. Defaults to None, and the first
            `input_dims` columns are selected.

            - variance:Optional[float] := The output kernel variance scaling parameter, usually
            denoted `\eta`. Defaults to 1.0. Must be positive.

            - weight_variance:Optional[Union[float, np.typing.NDArray[float]]]=1.0. The variance
            of the weight in the A.N.N. Can be specified as either a scalar or a matrix of
            appropriate size (same as the first input matrix). Optional.  Defaults to 1.0.

            - bias_variance:Optional[Union[float, np.typing.NDArray[float]]]=1.0. The variance of
            the biases in the A.N.N. Can be either a scalar or a matrix of appropriate size. When
            computing covariance matrices (single input) must be a N-length vector of biases for a
            `N \times M` input matrix. When computing cross-covariance matrices, must be an
            `N \times N` matrix. Optional.  Defaults to 1.0.

            - ARD:bool=False := A(utomatic)R(elevance)D(etermination) flag. Unused and raises
            an error if switched. Optional. Defaults to False.
        
        
        Returns:
        -------
        
            - K:np.typing.NDArray := When called with the `.full(X)` or the `.full(X, Y)` method,
            computes and returns the covariance or cross-covariance matrices.
            
            - diag(K):typing.NDArray := When called with the `.diag(X)` or `.diag(X, Y)` method
            computes and returns the diagonal of the covariance or cross-covariance matrices
    '''
    four_over_tau = 2./np.pi
    
    def __init__(self,input_dims:int, active_dims:typing.Union[list[int], int]=None,
                 variance:float=1.0,
                 weight_variance:typing.Union[float, np.typing.NDArray]=1.0,
                 bias_variance:typing.Union[float, np.typing.NDArray]=1.0, ARD:bool=False):
        super(MultiLayerPerceptronKernel, self).__init__(input_dims, active_dims=active_dims)
        self.variance = variance
        self.weight_variance = weight_variance
        self.bias_variance = bias_variance
        if ARD:
            raise NotImplementedError("Automatic Relevance Determination, not implemented")
        self.ARD = ARD

    
    def _comp_prod(self, X, X2=None):
        if X2 is None:
            return (at.math.square(X)*self.weight_variance).sum(axis=1)+self.bias_variance
        else:
            return (X*self.weight_variance).dot(X2.T)+self.bias_variance
    
    def diag(self, X):
        """Compute the diagonal of the covariance matrix for X."""
        X_prod = self._comp_prod(X)
        return self.variance*MultiLayerPerceptronKernel.four_over_tau*at.math.arcsin(X_prod/(X_prod+1.))
    
    def full(self, X, X2=None):
        if X2 is None:
            X_denom = at.math.sqrt(self._comp_prod(X)+1.)
            X2_denom = X_denom
            X2 = X
        else:
            X_denom = at.math.sqrt(self._comp_prod(X)+1.)
            X2_denom = at.math.sqrt(self._comp_prod(X2)+1.)
        XTX = self._comp_prod(X,X2)/X_denom[:,None]/X2_denom[None,:]
        return self.variance*MultiLayerPerceptronKernel.four_over_tau*at.math.arcsin(XTX)
    
def advi_inspect_ELBO(fit_data, show:bool=True):
    '''
        Utility method to investigate ADVI fit.
        Returns log-ELBO against iterations.
    '''
    _h_advi_hist = fit_data.hist
    advi_elbo = pd.DataFrame(
        {'$log-ELBO$': -np.log(_h_advi_hist),
         'n': np.arange(_h_advi_hist.shape[0])})
    fig = sns.lineplot(y='$log-ELBO$', x='n', data=advi_elbo)
    if show:
        plt.show()
    return fig

def select_subarray(df:pd.DataFrame, targets:list[tuple[str,str,str]], indicators:list[tuple[str,str,str]],
                   scaleX:bool=True, train_split:bool=False,
                    mappings:typing.Optional[dict[str, typing.Union[str,int, float]]]=None,
                    dummify:bool=False,
                   )->tuple[pd.DataFrame, pd.DataFrame]:
    X = df.loc[:,indicators]
    X = std_scale(X) if scaleX else X
    Y = df.loc[:, targets]
    m = Y.loc[~Y.isna().any(axis=1),:].index.intersection(X.loc[~X.isna().any(axis=1),:].index)
    X = X.loc[m, :]
    Y = Y.loc[m,:]
    Y=Y.replace(mappings) if mappings is not None else Y
    Y = pd.get_dummies(Y) if dummify else Y
    if train_split:
        X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y,test_size=.1 ,random_state=44)
        return X_train, X_test, Y_train, Y_test
    else:
        return X, Y

In [4]:
def tf_model(Xtrain, Xtest, Ytrain, Ytest, loss:str='categorical_crossentropy',
             optimizer:str='adadelta',
             layers:typing.Optional[list[tuple[int, str]]]=None,
             metrics=['accuracy'], summary:bool=True, epochs:int=10,
             factivation:str='sigmoid',batch:int=64, callbacks:list=[],
            verbosity:int=1):
    M = Xtrain.shape[1]
    input_layer = tf.keras.Input(shape=(M,))
    x = input_layer
    for neurons, activation in layers:
        x = tf.keras.layers.Dense(math.ceil(neurons), activation=activation)(x)

    x = tf.keras.layers.Dense(math.ceil(Ytrain.shape[1]), activation=factivation)(x)
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    model.compile(optimizer=optimizer, loss=loss, 
                metrics=metrics)
    if summary:
        model.summary()
    model.fit(Xtrain, Ytrain, epochs=epochs, validation_data = (Xtest, Ytest),
              batch_size=batch, callbacks=callbacks, verbose=verbosity)
    return model
# Multiple query utility
query_multiple = lambda df, col, multiquery : functools.reduce(lambda p,n: "{col} == '{p}' | ".format(p=p, col=col) + "{col} == '{n}".format(n=n, col=col), multiquery).split(" == '",1)[1]+"'"

In [5]:
def tf_autoencoder(Xtrain, Xtest, Ytrain, Ytest, loss:str='categorical_crossentropy',
             optimizer:str='adadelta',
             layers:typing.Optional[list[tuple[int, str]]]=None,
             metrics=['accuracy'], summary:bool=True, epochs:int=10,
             activation:str='sigmoid',batch:int=64 ):
    M = X.shape[1]
    input_layer = tf.keras.Input(shape=(M,))
    x = input_layer
    rev_layers = copy(layers)
    rev_layers.reverse()
    for neurons, activation in layers:
        x = tf.keras.layers.Dense(math.ceil(neurons), activation=activation)(x)
    encoder = x
    for neurons, activations in rev_layers:
        x = tf.keras.layers.Dense(math.ceil(neurons), activation=activation)(x)
    x = tf.keras.layers.Dense(math.ceil(Ytrain.shape[1]), activation='sigmoid')(x)
    decoder = x
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    model.compile(optimizer=optimizer, loss=loss, 
                metrics=metrics)
    if summary:
        model.summary()
    model.fit(Xtrain, Ytrain, epochs=epochs, validation_data = (Xtest, Ytest),
              batch_size=batch, verbose=1)
    return model

In [10]:
def load_data():
    with open('core_mappings.pickle', 'rb') as handle:
        core_mappings = pickle.load(handle)
    with open('r_core_mappings.pickle', 'rb') as handle:
        r_core_mappings = pickle.load(handle)
    df = pd.read_excel('evoos_processed.xlsx',header=2)
    return core_mappings,r_core_mappings,df
core_mappings, r_core_mappings, df = load_data()

#### Γκαουσιανές Διεργασίες (*Gaussian Processes*)

##### Binary Classification

In [64]:
X = std_scale(chem_indicators)
# X= chem_indicators
Y = cdf.loc[:, [("origin", "cultivation", 'wat')]]
m = Y.loc[~Y.isna().any(axis=1),:].index.intersection(X.loc[~X.isna().any(axis=1),:].index)
X = X.loc[m, :]
Y = Y.loc[m,:]
Y=Y.replace({"Yes":1, "No":0})

he coregionalized regression model relies on the use of $\color{firebrick}{\textbf{multiple output kernels}}$ or $\color{firebrick}{\textbf{vector-valued kernels}}$ (Álvarez, Rosasco and Lawrence, 2012), of the following form:

$ \begin{align*} {\bf B}\otimes{\bf K} = \left(\begin{array}{ccc} B_{1,1}\times{\bf K}({\bf X}_{1},{\bf X}_{1}) & \ldots & B_{1,D}\times{\bf K}({\bf X}_{1},{\bf X}_{D})\\ \vdots & \ddots & \vdots\\ B_{D,1}\times{\bf K}({\bf X}_{D},{\bf X}_{1}) & \ldots & B_{D,D}\times{\bf K}({\bf X}_{D},{\bf X}_{D}) \end{array}\right) \end{align*} $.

In the expression above, ${\bf K}$ is a kernel function, ${\bf B}$ is a regarded as the coregionalization matrix, and ${\bf X}_i$ represents the inputs corresponding to the $i$-th output.

Notice that if $B_{i,j} = 0$ for $i \neq j$, then all the outputs are being considered as independent of each other.

To ensure that the multiple output kernel is a valid kernel, we need the $\bf K$ and ${\bf B}$ to be to be valid. If $\bf K$ is already a valid kernel, we just need to ensure that ${\bf B}$ is positive definite. The last is achieved by defining ${\bf B} = {\bf W}{\bf W}^\top + {\boldsymbol \kappa}{\bf I}$, for some matrix $\bf W$ and vector ${\boldsymbol \kappa}$.

In GPy, a multiple output kernel is defined in the following way:


Με τυπικούς πυρήνες (γραμμικός, περιοδικός, συνημιτονοειδής, ακτινωτής βάσης κ.λ.π) η επίλυση οι προβλέψεις είναι σταθερά χαμηλης πιθανότητας, κάτι που καταδεικνύει οτι το μοντέλο δεν έχει επιλύσει κάποια σχέση μεταξύ των χημικών παραμέτρων και το δεδομένων εξόδου, αλλά προβλέπει σταθερά "αρνητικά" για στη πλειοψηφία των περιπτώσεων οι ελιές δεν ποτίζονται. Με την εφαρμογή πυρήνα Νευρωνικού Δικτύου το πρόβλημα επιλύεται, δηλαδή έχουμε υψηλή σχετικά ακρίβεια και ποικιλία προβλήψεων, ιδιαίτερα στην μειωψηφική κατηγορία (1,0). Αυτό φαίνεται να υπονοεί ισχυρά μη-γραμμική σχέση μεταξύ χημικών παραμέτρων και ύδρευσης, κάτι για το οποίο έχουμε είδη ισχυρές ενδήξεις. Ο πυρήνας arcsin είναι:
$$
k(x,\ y)\ =\ \eta^2 \frac2\pi\arcsin\Bigg( \dfrac{\sigma_w^2x^Ty+\sigma_b^2}{\sqrt{\sigma_w^2x^Tx+\sigma_b^2+1}\sqrt{\sigma_w^2y^Ty+\sigma_b^2+1}}  \Bigg)
$$

Αυτός ο πυρήνας (*Deep Neural Networks as Gaussian Processes, J. Lee et all, 2018*) (*Large-Margin Classification in Infinite Neural Networks, Youngmin Cho et. al 2010*) σχεδιάστηκε ώστε να μιμείται τις μη-γραμμικές διεργασίες σε Νευρωνικό Δίκτυο. Βασίζεται στο Κεντρικό Οριακό Θεώρημα και αποδεικνύεται ότι Νευρωνικό Δίκτυο μοναδικό κρυφού layer γίνεται ισοδυναμεί με GP όταν τα συναπτικά βάροι και τα biases είναι κανονικά κατανεμημένα και ο αριθμός το νευρώνων απειρίζεται. Μπορεί να θεωρηθεί ως μη-γραμμικός μετασχηματισμός των δεδομένων που ακολουθείται από γραμμική παλινδρόμηση 

Μπορεί να απλοποιηθεί στη περιπτωσή μας ως:
$$
\require{cancel}
\mathbf{K}(X,X^T)=\eta^2\frac{2}{\pi}\arcsin{\Big(\dfrac{\sigma^2_wXX^Τ+ \cancelto{0}{\sigma_b^2} }{\sigma_w^2XX^Τ+ \cancelto{0}{\sigma_b^2} +\frac 1 {\sigma^2_w} } \Big)}
$$

Ισοδυναμεί ζύγιση όλων των  μεταβλητών με τον παράγοντα $\sigma^2_w$, κανονικοποίηση, και διαβίβαση μέσο της συνάρτησης $\arcsin:[-1,+1]\rightarrow [-1,+1]$

Καταλήξαμε σε μοντέλο δυαδικού classification μορφής:
$$
\
$$

In [ ]:
with pymc.Model() as binary_classification_model:
    N,M = X_train.shape
    σ=pymc.Normal('s', mu=2.0, sigma=0.5)
    σ_w = pymc.MvNormal('σ_w', mu=[10.0]*M, cov=σ*np.eye(M), shape=M)
    σ_b = pymc.Normal('σ_b',mu=10.0, sigma=2.0)
    _η = pymc.Exponential('_η', lam=2.0)
    ϵ = 1e-9
    η = pymc.Deterministic('η', _η+ϵ)
    K = MultiLayerPerceptronKernel(63, variance=η, weight_variance=σ_w, bias_variance=σ_b)
    μ = pymc.gp.mean.Zero()
    gp = pymc.gp.Latent(mean_func=μ, cov_func=K)
    _f = gp.prior('_f', X=X_train.values)
    f = pymc.Deterministic('f', at.math.sigmoid(_f))
    y = pymc.Bernoulli('y', p = f, observed=Y_train.values[:,0])

##### Multiclass Unilabel Classification

In [43]:
X_train,X_test, Y_train, Y_test = select_subarray(cdf, targets =[('origin', 'location', 'loc')], indicators=chem_indicators.columns,
                                                train_split=True, mappings=r_core_mappings, dummify=False)
Y_train, Y_test =  Y_train.astype(float)-1, Y_test.astype(float)-1

In [45]:
# MGP withought coregionalization
with pymc.Model() as categorical_simple_model:
    factors=5
    N,M=X_train.shape
    gps=[]
    fs=[]
    G = 1e7
    η = 10.0
    σ_b= pymc.Normal(f'σ_b', mu=0.0, sigma=0.1)
    for factor,loc in  list(core_mappings[('origin', 'location', 'loc')].items()):
        μ = pymc.gp.mean.Constant(0)
        σ_c = pymc.LogNormal(f'σ_c_{loc}', mu=2.30, sigma=0.25)
        σ_w = pymc.MvNormal(f'σ_w_{loc}', mu=3.0*np.ones(M), cov=σ_c*np.eye(M), shape=M)
        κ =  MultiLayerPerceptronKernel(M, variance=η**2, bias_variance=σ_b**2, weight_variance=σ_w**2)
        gp = pymc.gp.Latent(mean_func=μ, cov_func=κ)
        gps.append(gp)
        f = gp.prior(f'f_{loc}'.format(loc=loc),
            X=x_tensor)
        fs.append(f)
    f = pymc.Deterministic('f', at.stack(*fs).T)
    p = pymc.Deterministic('p', at.nnet.softmax(f, axis=1))
    # y = pymc.Multinomial('y', p=p ,observed=Y_train.values, n=1)
    y = pymc.Categorical('y', p=p, observed=y_tensor)

## Intrensic Coregionalization GPs

In [ ]:
X, Y = select_subarray(cdf, targets =[('origin', 'location', 'loc')], indicators=chem_indicators.columns,
                                                train_split=False, mappings=r_core_mappings, dummify=False)
Y = Y.astype(float)-1.0

In [ ]:
factors = len(np.unique(Y.values))
N,M = X.values.shape
idx = np.concatenate([np.zeros(N)[:,None]]+[i*np.ones(N)[:,None] for i in range(1,factors)])
y =Y
x= pd.DataFrame(data= np.concatenate([idx, np.tile(X.values, (factors,1))],axis=1),columns = pd.Index([("", "", "GP")]).append(X.columns))

In [ ]:
ADVI = True
MINIBATCH_ADVI_SIZE=100
N_ITERS = 2500
x_tensor =aesara.shared(x.values)
y_tensor = aesara.shared(y.values[:,0])
map_tensor_batch = {
    y_tensor:pymc.Minibatch(y.values[:,0], MINIBATCH_ADVI_SIZE),
    x_tensor:pymc.Minibatch(x.values, MINIBATCH_ADVI_SIZE),
    }

In [53]:
# with pymc.Model() as categorical_ICM:
#     LOWER_BOUND_NON_NEGATIVE = 1e-4
#     σ_n = pymc.TruncatedNormal('σ_n', mu=10.0, sigma=2, lower=LOWER_BOUND_NON_NEGATIVE, shape=x.shape[0], initval=(np.ones(x.shape[0])*1.0))
#     k = pymc.TruncatedNormal('k', mu=10.0, sigma=2,lower=LOWER_BOUND_NON_NEGATIVE ,shape=factors)
#     W = (-1)*pymc.TruncatedNormal('W', mu=10.0, sigma=2.0,lower=LOWER_BOUND_NON_NEGATIVE, shape=(factors,2) )
#     σ_b = pymc.TruncatedNormal('σ_b', mu=3.0, sigma=1.0, lower=LOWER_BOUND_NON_NEGATIVE)
#     σ_w = pymc.TruncatedNormal('σ_w',mu=3.0, sigma=0.8,lower=LOWER_BOUND_NON_NEGATIVE ,shape=x.shape[1])
#     G = 1
#     η = pymc.TruncatedNormal('η',mu=10, sigma=3.0, lower=LOWER_BOUND_NON_NEGATIVE)
#     K = MultiLayerPerceptronKernel(x.shape[1],active_dims=[i for i in range(1,x.shape[1])],
#                                   variance = G*(η+1.0), bias_variance = σ_b**2, weight_variance=σ_w**2)
#     B = pymc.gp.cov.Coregion(X.shape[1], kappa=k, W=W, active_dims=[0])
#     k_noise = pymc.gp.cov.WhiteNoise(σ_n)
#     κ = (K+k_noise)*B
#     μ = pymc.gp.mean.Constant(c=0)
#     gp = pymc.gp.Latent(mean_func=μ, cov_func=κ)
#     _f = gp.prior('_f', X=x_tensor)
#     f = pymc.Deterministic('f', _f.reshape((-1, factors)))
#     p = pymc.Deterministic('p', at.nnet.softmax(f, axis=1))
#     y_obs = pymc.Categorical('y_obs',p=p ,observed=y_tensor)

/media/alexander-fyrogenis/Elements/Διδακτορικό/Olive Oil/notebooks/venv/lib/python3.10/site-packages/pymc/gp/cov.py:99: UserWarning: Only 63 column(s) out of 64 are being used to compute the covariance function. If this is not intended, increase 'input_dim' parameter to the number of columns to use. Ignore otherwise.
  warnings.warn(


In [55]:
# # WIP Needs re-thought
# with pymc.Model() as categorical_ICM:
#     LOWER_BOUND_NON_NEGATIVE = 1e-4
#     σ_n = pymc.TruncatedNormal('σ_n', mu=10.0, sigma=2, lower=LOWER_BOUND_NON_NEGATIVE, shape=M)
#     k = pymc.TruncatedNormal('k', mu=100.0, sigma=20.0,lower=LOWER_BOUND_NON_NEGATIVE ,shape=factors)
#     W = (-1)*pymc.TruncatedNormal('W', mu=100.0, sigma=20.0,lower=LOWER_BOUND_NON_NEGATIVE, shape=(factors,5) )
#     weights=[]
#     for loc in r_core_mappings[('origin','location','loc')].keys():
#         σ_b = pymc.MvNormal(f'σ_b_{loc}', mu=3.0, sigma=1.0, lower=LOWER_BOUND_NON_NEGATIVE)
#     σ_w = pymc.TruncatedNormal('σ_w',mu=3.0, sigma=0.8,lower=LOWER_BOUND_NON_NEGATIVE ,shape=x.shape[1]-1)
#     G = 1
#     η = pymc.TruncatedNormal('η',mu=3.0, sigma=1.0, lower=LOWER_BOUND_NON_NEGATIVE)
#     K = MultiLayerPerceptronKernel(63,
#                                   variance = G*(η**2), bias_variance = σ_b, weight_variance=σ_w)
#     B = pymc.gp.cov.Coregion(1, kappa=k, W=W)
#     k_noise = pymc.gp.cov.WhiteNoise(σ_n)
#     κ = pymc.gp.cov.Kron([B,K])
#     μ = pymc.gp.mean.Constant(c=0)
#     gp = pymc.gp.Latent(mean_func=μ, cov_func=κ)
#     _f = gp.prior('_f', X=x_tensor)
#     f = pymc.Deterministic('f', _f.reshape((-1, factors)))
#     p = pymc.Deterministic('p', at.nnet.softmax(f, axis=1))
#     y_obs = pymc.Categorical('y_obs',p=p ,observed=y_tensor)

In [45]:
# with pymc.Model() as categorical_ICM:
#     LOWER_BOUND_NON_NEGATIVE = 1e-4
#     σ_n = pymc.TruncatedNormal('σ_n', mu=10.0, sigma=2, lower=LOWER_BOUND_NON_NEGATIVE, shape=M)
#     k = pymc.TruncatedNormal('k', mu=100.0, sigma=20.0,lower=LOWER_BOUND_NON_NEGATIVE ,shape=factors)
#     W = (-1)*pymc.TruncatedNormal('W', mu=100.0, sigma=20.0,lower=LOWER_BOUND_NON_NEGATIVE, shape=(factors,2) )
#     η = pymc.LogNormal('η', mu=.0, sigma=0.25)
#     G = 3.0
#     λ = pymc.MvNormal
#     K = η**2*pymc.gp.cov.ExpQuad(63,ls=λ)
#     B = pymc.gp.cov.Coregion(1, kappa=k, W=W)
#     k_noise = pymc.gp.cov.WhiteNoise(σ_n)
#     κ = pymc.gp.cov.Kron([B,K])
#     μ = pymc.gp.mean.Constant(c=0)
#     gp = pymc.gp.Latent(mean_func=μ, cov_func=κ)
#     _f = gp.prior('_f', X=x_tensor)
#     f = pymc.Deterministic('f', _f.reshape((-1, factors)))
#     p = pymc.Deterministic('p', at.nnet.softmax(f, axis=1))
#     y_obs = pymc.Categorical('y_obs',p=p ,observed=y_tensor)

### General GP

In [ ]:
design_matrix= dict(
    categorical = dict(
        variables=list(),
        kernels=list(),
    ),
    binary=dict( 
        variables=list(),
        kernels=list()
    ),
    
    continuous=dict(
        variables=list(),
        kernels=list())
)

In [52]:
cdf.loc[:,['origin'].columns

MultiIndex([('origin',    'location',           'zn'),
            ('origin',    'location',          'loc'),
            ('origin',    'cultivar',         'uvar'),
            ('origin',    'cultivar',         'ucul'),
            ('origin',    'cultivar',          'kol'),
            ('origin',    'cultivar',          'kor'),
            ('origin',    'cultivar',          'lad'),
            ('origin',    'cultivar',          'adr'),
            ('origin',    'cultivar',          'thr'),
            ('origin',    'cultivar',          'hon'),
            ('origin',    'cultivar',          'kal'),
            ('origin',    'cultivar',          'man'),
            ('origin',    'cultivar',          'chi'),
            ('origin',    'cultivar',          'kou'),
            ('origin',    'cultivar',          'cre'),
            ('origin',    'cultivar',          'agr'),
            ('origin',    'cultivar',          'dap'),
            ('origin',    'cultivar',          'hou'),
          

In [42]:
binary=[
            ('origin', 'cultivation','wat'),
            ('origin', 'cultivation','org_b'),
            # ('origin',  'production',       'mat_pa'),
            # ('origin',  'production',       'mat_pi'),
            # ('origin',  'production',        'mat_p'),
            # ('origin',  'production',        'mat_m'),
            ('origin',  'production',        'centr'),
            ('origin',  'production',     'watadd_b')
       ]

continuous = [
            ('origin',  'production',       'kbtemp'),
            ('origin',  'production',       'kntemp'),
            ('origin',  'production',        'stemp'),
            ('origin',  'production',         'doil'),
            ]
categorical =[
            ('origin',    'location',          'loc'),
            ('origin', 'cultivation',     'altitude')
]
categorical_hot = [
            ('origin',    'cultivar',          'kol'),
            ('origin',    'cultivar',          'kor'),
            ('origin',    'cultivar',          'lad'),
            ('origin',    'cultivar',          'adr'),
            ('origin',    'cultivar',          'thr'),
            ('origin',    'cultivar',          'hon'),
            ('origin',    'cultivar',          'kal'),
            ('origin',    'cultivar',          'man'),
            ('origin',    'cultivar',          'chi'),
            ('origin',    'cultivar',          'kou'),
            ('origin',    'cultivar',          'cre'),
            ('origin',    'cultivar',          'agr'),
            ('origin',    'cultivar',          'dap'),
            ('origin',    'cultivar',          'hou'),
            ('origin',    'cultivar',          'les'),
            ('origin',    'cultivar',          'arm'),
            ('origin',    'cultivar',          'pat'),
            ('origin',    'cultivar',          'amf'),
            ]

In [43]:
X=cdf.loc[:,['chemical']]
Y=cdf.loc[:,['origin']]
X_train,X_test,Y_train,Y_test = sklearn.model_selection.train_test_split(X,Y)

In [44]:
select_valid_data = lambda target_label: Y.loc[~Y.loc[:,[target_label]].isna().any(axis=1),:].index.intersection(X.loc[~X.isna().any(axis=1),:].index)

In [47]:
numpyro.set_platform('gpu')
with pymc.Model() as evoos_model:
    M = X_train.shape[1]
    # Binary
    for binary_var in binary:
        
        x = X_train.loc[X_train.index.intersection(select_valid_data(binary_var)),:].values
        _y = Y_train.loc[Y_train.index.intersection(select_valid_data(binary_var)),[binary_var] ]
        y = _y.replace(r_core_mappings)
        
        m = pymc.gp.mean.Constant(c=0)
        ls= pymc.Normal(f'l_{binary_var}', mu=3.0, sigma=1.2,shape=M)
        k = pymc.gp.cov.ExpQuad(X_train.shape[1], ls**2)
        
        gp = pymc.gp.Latent(mean_func=m,cov_func=k)
        f = gp.prior(f'f_{binary_var}', X=x)
        p= pymc.Deterministic(f'p_{binary_var}', f.T)
        
        y_obs = pymc.Bernoulli(f'y_obs_{binary_var}',p=p ,observed=y)
    

In [ ]:
with evoos_model:
    idata = sample_numpyro_nuts(draws=500, tune=1000, chains=2, postprocessing_backend='cpu')